## Homework 10: GPUs

## Due Date: April 26, 2023, 11:59pm

#### Firstname Lastname: 

#### E-mail: 

#### Enter your solutions and submit this notebook

---

**Problem 1 (100p)**


Write two programs which will be able to run in parallel on a GPU, one using Numba/CUDA (50p), one using PyOpenCL (50p).


Each program will:

- draw two random vectors $\vec u$ and $\vec v$ from $[0,1]^N$ where $N = 10^7$;


- calculate and output similarity between $\vec u$ and $\vec v$.




The similarity between two vectors $\vec u$ and $\vec v$ is defined here as a `cosine` value of the angle between them $\measuredangle \left( \vec u, \vec v \right)$. That is, the program returns: 

$$\cos \left( \measuredangle \left( \vec u, \vec v \right) \right).$$


Note that the output is a real value and must belong to $[-1, 1]$.

## cuda 

In [10]:
# %%writefile cuda01.py

from __future__ import division
from numba import cuda
import numpy as np
import math

# CUDA kernel
@cuda.jit
def my_kernel(input_array, output_array):
    
    tx = cuda.threadIdx.x
    bx = cuda.blockIdx.x
    bw = cuda.blockDim.x
    print("tx")
    print("bx")
    print("bw")
    
    index = tx + bx * bw
    
    a = output_array[index][0]*output_array[index][1]
    b = output_array[index][0]*output_array[index][1]
    c = output_array[index][0]*output_array[index][1]
    
    input_array[index] = index * 10
    print("i, t, b, w:", index, tx, bx, bw)
        
        
# Host code   
n=100
input_array = np.random.uniform(0,1,(n,2))
output_array = np.zeros(n)
cuda.select_device(0)
data = np.ones(256)
threadsperblock = 16 
blockspergrid = math.ceil(data.shape[0] / threadsperblock)

my_kernel[blockspergrid, threadsperblock](data)
print("\ndata:\n", data)

0.0

In [24]:
from numba import cuda
from numba.cuda.random import create_xoroshiro128p_states, xoroshiro128p_uniform_float32
import numpy as np

@cuda.jit
def cosine_similarity(rng_states, iterations, out):
    """Find the maximum value in values and store in result[0]"""
    thread_id = cuda.grid(1)

    # Compute pi by drawing random (x, y) points and finding what
    # fraction lie inside a unit circle
    a,b,c=0 ,0 ,0
    for i in range(iterations):
        x = xoroshiro128p_uniform_float32(rng_states, thread_id)
        y = xoroshiro128p_uniform_float32(rng_states, thread_id)
        a=a+(x*y)
        b=b+(x**2)
        c=c+(y**2) ## could try to do with like an array 

    out[thread_id] = a/(b**(1/2)*c**(1/2))
n = 10e7
data = np.ones(256)
iterations = n//data.shape[0]
print(iterations)
threads_per_block = 64
blocks = math.ceil(data.shape[0] / threads_per_block)
rng_states = create_xoroshiro128p_states(threads_per_block * blocks, seed=1)
out = np.zeros(threads_per_block * blocks, dtype=np.float32)

cosine_similarity[blocks, threads_per_block](rng_states, iterations, out)
print('cosine similiarty:', out.mean())

390625.0


/home/buzgalbraith/miniconda3/envs/prob_stats_2/lib/python3.10/site-packages/numba/cuda/dispatcher.py:488: NumbaPerformanceWarning: Grid size 4 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
/home/buzgalbraith/miniconda3/envs/prob_stats_2/lib/python3.10/site-packages/numba/cuda/cudadrv/devicearray.py:885: NumbaPerformanceWarning: Host array used in CUDA kernel will incur copy overhead to/from device.
  warn(NumbaPerformanceWarning(msg))


cosine similiarty: 0.75003445
